### Data preparation

Importing the libraries we will use to solve the linear programming challenge

In [1]:
import numpy as np
import math
import pandas as pd
import setup
from docplex.mp.model import Model

Reading the xslx files with all the data of the problem

In [2]:
dfdemand = pd.read_excel('data_or_test.xlsx', sheet_name='Demand')
dfMarkup = pd.read_excel('data_or_test.xlsx', sheet_name='Markup')
dfLastmile = pd.read_excel('data_or_test.xlsx', sheet_name='Last Mile')
dfStock = pd.read_excel('data_or_test.xlsx', sheet_name='Stock')
dfIniStock = pd.read_excel('data_or_test.xlsx', sheet_name='Initial Stock')
dfTransfers = pd.read_excel('data_or_test.xlsx', sheet_name='Transfers')
dfInbound = pd.read_excel('data_or_test.xlsx', sheet_name='Inbound')
dfProduct = pd.read_excel('data_or_test.xlsx', sheet_name='Product')

Visualizing the data as a dataframe and checking if it's in the correct format 

In [3]:
dfdemand.head()

,State,Seller,Product,Week,Demand (unit),Price
0,ST1,S1,P1,W1,668,1449.19
1,ST1,S1,P1,W2,887,1983.44
2,ST1,S1,P1,W3,499,1386.48
3,ST1,S1,P1,W4,281,889.35
4,ST1,S2,P2,W1,144,2115.55


Defining the list of values in which our variables will span. 

In [4]:
list((dfTransfers["Category"]).unique())

['C1', 'C2', 'C3', 'C4', 'C5']

In [5]:
days = list(range(1,4 )) # DEBERIA SER 29 CAMBIAR!!!!!!!!!!!!!!!!!!!!!!!!!!!
weeks =  list((dfdemand["Week"]).unique())
fcenters = list((dfTransfers["FC"]).unique())
sellers = list((dfdemand["Seller"]).unique())
states = list((dfdemand["State"]).unique())
products = list((dfdemand["Product"]).unique())
categories = list((dfTransfers["Category"]).unique())

Example:

In [6]:
print(states)

['ST1', 'ST2', 'ST3', 'ST4', 'ST5', 'ST6', 'ST7', 'ST8', 'ST9', 'ST10']


# The mathematical model

### Variables

We define the CPLEX model we will be using

In [7]:
mdl = Model(name="modelEcoMole")

 When creating variables, we will use the following type format:
 B,I,C,N,S or type short names (e.g.: binary, integer, continuous, semicontinuous, semiinteger)

stockAtFC represents the amount of stock that a given fullfilment center has at a given day of a given product. Therefore it must be any non negative integer.

In [8]:
stockAtFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(stockAtFC)

2700

transferBetweenFCs represents the amount of products that a given fullfilment center transfers to another fullfilment center at a given day of a given product. Therefore it must be any non negative integer.

In [9]:
transferBetweenFCs = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, fcenters, days, products])
len(transferBetweenFCs)

8100

In [10]:
deliveryToStateFromFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, states, days, products])
len(deliveryToStateFromFC)

27000

In [11]:
amountInboundAtFCProd = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(amountInboundAtFCProd)

2700

### Auxiliar functions

In [12]:
def giveCategoryOfProduct(p):
    return(list(dfProduct[dfProduct["Product"] == p]["Category"])[0])

In [13]:
giveCategoryOfProduct("P2")

'C5'

In [14]:
def giveSellerOfProduct(p):
    return(list(dfIniStock[dfIniStock["Product"] == p]["Seller"])[0])

In [15]:
giveSellerOfProduct("P124")

'S24'

In [16]:
def giveDemandOfProduct(p, sta, w):
    return(list(dfdemand[(dfdemand["Product"] == p)&(dfdemand["State"] == sta)&(dfdemand["Week"] == w)]["Demand (unit)"])[0])

In [17]:
giveDemandOfProduct("P3", "ST1", "W3")

397

In [18]:
def givePriceOfProduct(p, sta, w):
    return(list(dfdemand[(dfdemand["Product"] == p)&(dfdemand["State"] == sta)&(dfdemand["Week"] == w)]["Price"])[0])

In [19]:
givePriceOfProduct("P2","ST1", "W4")

1312.53

In [20]:
def giveLastMileCostOfProduct(p, sta, fc):
    c = giveCategoryOfProduct(p)
    return(list(dfLastmile[(dfLastmile["Category"] == c)&(dfLastmile["State"] == sta)&(dfLastmile["FC"] == fc)]["Cost (per unit)"])[0])

In [21]:
giveLastMileCostOfProduct("P2", "ST4", "FC3")

34.0

In [22]:
def giveWeekOfDay(d):
    w = math.ceil((d/28 * 4))
    weekAsString = "W"+str(w)
    return(weekAsString)

In [23]:
giveWeekOfDay(11)

'W2'

In [24]:
def giveCostOfInbound(s,c,fc):
    return(list(dfInbound[(dfInbound["Seller"] == s) & (dfInbound["Category"] == c) & (dfInbound["FC"] == fc) ]["Cost (per unit)"])[0])

In [25]:
giveCostOfInbound("S2","C3","FC2")

30.25

In [83]:
def giveTimeOfInbound(s,c,fc):
    return(list(dfInbound[(dfInbound["Seller"] == s) & (dfInbound["Category"] == c) & (dfInbound["FC"] == fc) ]["Time (days)"])[0])

In [85]:
giveTimeOfInbound("S2","C4","FC2")

3

In [61]:
def giveCostOfStorage(p, fc):
    return(list(dfStock[(dfStock["Product"] == p)&(dfStock["FC"] == fc)]["Cost (per day and unit)"])[0])

In [62]:
giveCostOfStorage("P5", "FC2")

5.36

In [28]:
def giveCostOfTransfer(fc1, fc2, c):
    return(list(dfTransfers[(dfTransfers["FC"] == fc1)&(dfTransfers["FC.1"] == fc2) &(dfTransfers["Category"] == c) ]["Cost (per unit)"])[0])

In [29]:
giveCostOfTransfer("FC3", "FC1", "C4")

10.37

In [79]:
def giveTimeOfTransfer(fc1, fc2, c):
    return(list(dfTransfers[(dfTransfers["FC"] == fc1)&(dfTransfers["FC.1"] == fc2) &(dfTransfers["Category"] == c) ]["Time (days)"])[0])

In [82]:
giveTimeOfTransfer("FC3", "FC1", "C4")

3

In [30]:
def giveMarkUpOfSeller(s):
    return(list(dfMarkup[(dfMarkup["Seller"] == s)]["Markup (%)"])[0])

In [31]:
giveMarkUpOfSeller("S4")

7

In [32]:
def giveInitialStock(p, fc):
    return(list(dfIniStock[(dfIniStock["Product"] == p)&(dfIniStock["FC"] == fc)]["Quantity (unit)"])[0])

In [33]:
giveInitialStock("P190", "FC3")

162

### Objective function and related restrictions

In [34]:
InboundCosts = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for p in products:
            c = giveCategoryOfProduct(p)
            s = giveSellerOfProduct(p)
            costThisCSFC = giveCostOfInbound(s,c,fc)
            InboundCosts = InboundCosts + costThisCSFC * amountInboundAtFCProd[fc,d,p]

In [63]:
StorageCost = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for p in products:
            costThisP = giveCostOfStorage(p, fc)
            StorageCost = StorageCost + costThisP * stockAtFC[fc,d,p]

In [36]:
TransfersCosts = 0
for d in days:
    #print(d)
    for fc1 in fcenters:
        for fc2 in fcenters:
            if(fc1 != fc2):
                for p in products:
                    c = giveCategoryOfProduct(p)
                    costThisTran = giveCostOfTransfer(fc1, fc2, c)
                    TransfersCosts = TransfersCosts + costThisTran * transferBetweenFCs[fc1,fc2,d,p]

Los costos de last mile no usan demanda porque la restriccion de la demanda esta en otro lado

In [37]:
LastMileCosts = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for st in states:
            for p in products:
                costThisLastmile = giveLastMileCostOfProduct(p, st, fc)
                TransfersCosts = TransfersCosts + costThisLastmile * deliveryToStateFromFC[fc,st,d,p]

In [64]:
TotalCosts = InboundCosts + StorageCost + TransfersCosts + LastMileCosts

Note that revenue from markup is not a variable we can change. ()

Therefore defining: Profit = Revenue - Costs 

is only dependant on Costs. That means that the solutions to maximize(Profit) are equivalent to the ones that minimize(Costs)

In [39]:
revenue = 0
for w in weeks:
    for p in products:
        for st in states:
            demandaThis = giveDemandOfProduct(p, st, w)
            priceThis = givePriceOfProduct(p, st, w)
            s = giveSellerOfProduct(p)
            markupThis = giveMarkUpOfSeller(s)
            revenue = revenue + (markupThis/100) * priceThis * demandaThis

In [40]:
print(revenue)

1369020027.1043003


In [71]:
Profit = revenue - TotalCosts

In [72]:
mdl.maximize(Profit)

### Stock restrictions

We set the initial stock

In [73]:
for fc in fcenters:
    for p in products:
        stockAtFC[fc, 1, p] = giveInitialStock(p, fc) #On day 1 we have the initial stock

The stock at a given day is the stock the stock of yesterday + stock arrived - the stock that departed.
The stock can arrive or departure either from sellers or other fullfilment centers.

In [87]:
for d in days:
    if(d!=1):
        for fc1 in fcenters:
            for p in products:
                c = giveCategoryOfProduct(p)
                s = giveSellerOfProduct(p)
                arrivals = 0
                departures = 0
                for fc2 in fcenters:
                    if(fc1!=fc2):
                        potentialTimeArrival = giveTimeOfTransfer(fc2, fc1, c)
                        dprev = d - potentialTimeArrival
                        if(dprev>=1):
                            arrivals = arrivals + transferBetweenFCs[fc2,fc1,dprev,p]
                        departures = departures + transferBetweenFCs[fc1,fc2,d,p] 
                
                potentialTimeInbound = giveTimeOfInbound(s,c,fc1)
                dprev2 = d - potentialTimeInbound
                if(dprev2>=1):
                    arrivals = arrivals + amountInboundAtFCProd[fc1,d,p]
                for st in states:
                    departures = departures + deliveryToStateFromFC[fc1, st, d, p]
                stockAtFC[fc1, d, p] = stockAtFC[fc1, d-1, p] + arrivals - departures

In [88]:
msol = mdl.solve()

In [89]:
msol.get_value(Profit)

1368699151.2743003

In [100]:
msol.get_value(stockAtFC[("FC1",2,"P1")])

101

In [101]:
msol.get_value(TotalCosts)

320875.8299999998

In [102]:
msol.get_value(StorageCost)

320875.8299999998